In [ ]:
!pip install flask pyngrok==4.1.1
!ngrok authtoken '1u5a8Je6P84l0dQN0VimEevIUJI_2BkrrVaULvsosiE4fDUL4'
!pip install flask-ngrok




  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=f4ba4e2532b5f531d726db51a96c6bdc044a5c77fec6d97240fc6d733b9977e5
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from flask import Flask, render_template, request, redirect, g
from flask_ngrok import run_with_ngrok
import sqlite3
import os

#mount drive
from google.colab import drive
drive.mount('/content/drive')
templates_folder = '/content/drive/MyDrive/myproject/templates'
static_folder = '/content/drive/MyDrive/myproject/static'

app = Flask(__name__, template_folder=templates_folder,static_folder=static_folder)
run_with_ngrok(app)

# Database operations
db_folder = "/content/drive/MyDrive/myproject/"
DATABASE = db_folder+'mydatabase.db'

def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        db = g._database = sqlite3.connect(DATABASE)
        #query
        create_table_query = '''
        CREATE TABLE IF NOT EXISTS users (
          id INTEGER PRIMARY KEY AUTOINCREMENT,
          username TEXT NOT NULL,
          password TEXT NOT NULL
        )
        '''
        db.execute(create_table_query)
        insert_query = '''
        INSERT INTO users (username, password)
        SELECT ?, ?
        WHERE NOT EXISTS (
          SELECT * FROM users WHERE username = ?
        )
        '''
        user_data = ('alif2','12345','alif2')
        db.execute(insert_query,user_data)
        db.commit()
       # Product table creation
        create_product_table_query = '''
        CREATE TABLE IF NOT EXISTS products (
          id INTEGER PRIMARY KEY AUTOINCREMENT,
          name TEXT NOT NULL,
          description TEXT NOT NULL,
          price REAL NOT NULL
        )
        '''
        db.execute(create_product_table_query)
        # Insert dummy products
        products_data = [
            ('Product 1', 'Description for product 1', 9.99),
            ('Product 2', 'Description for product 2', 19.99),
            ('Product 3', 'Description for product 3', 29.99),
            ('Product 4', 'Description for product 4', 39.99),
            ('Product 5', 'Description for product 5', 49.99),
        ]
        for product in products_data:
            db.execute('INSERT INTO products (name, description, price) VALUES (?, ?, ?)', product)
        db.commit()
    return db

@app.teardown_appcontext
def close_connection(exception):
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

@app.route('/')
def home():
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM users')
    users = cursor.fetchall()
    cursor.execute('SELECT * FROM products')
    products = cursor.fetchall()
    return render_template('index.html', users=users, products=products)




@app.route('/list_users')
def list_users():
    cursor.execute('SELECT * FROM users')
    users = cursor.fetchall()
    return render_template('list_users.html', users=users)

# Rest of your Flask routes ...

@app.route('/contact')
def contact():
  return render_template('contact.html')

@app.route('/hi/')
def hi(name):
  return render_template('hi.html', name = name)

@app.route('/get_form')
def get_form():
  return render_template('get_form.html')

@app.route('/process', methods=['GET','POST'])
def process():
  mass = request.form['mass']
  velocity = request.form['velocity']
  ke = 0.5* float(mass)*float(velocity)**2
  return render_template('result.html', ke=ke)

@app.route('/get_form_pe')
def get_form_pe():
  return render_template('get_form_pe.html')

@app.route('/process_pe', methods=['GET','POST'])
def process_pe():
  mass = request.form['mass']
  gravity = request.form['gravity']
  height = request.form['height']
  pe = float(mass)*float(gravity)*float(height)
  return render_template('result2.html', pe=pe)

@app.route('/list_student')
def list_student():
  return render_template('list_student.html', students=students)

@app.route('/add_student/<info>')
def add_student():
  info = info.split()
  students.append(info)
  return redirect ('/list_student')

@app.route('/delete_student/<info>')
def delete_student():
  info = info.split()
  students.append(info)
  return redirect ('/list_student')

@app.route('/find_student/<name>')
def find_student(name):
  for student in students:
    if name == student[0]:
      return (name)
  return('Record not found')

@app.route('/update_user', methods=['POST'])
def update_user():
    data = request.get_json()
    id = data['id']
    username = data['username']
    password = data['password']
    conn = get_db()
    cursor = conn.cursor()
    # Update the user in the database here
    update_query = '''
    UPDATE users
    SET username = ?, password = ?
    WHERE id = ?
    '''
    cursor.execute(update_query, (username, password, id))
    conn.commit()
    return 'OK', 200

@app.route('/delete_user', methods=['POST'])
def delete_user():
    data = request.get_json()
    id = data['id']
    conn = get_db()
    cursor = conn.cursor()
    # Delete the user from the database here
    delete_query = '''
    DELETE FROM users
    WHERE id = ?
    '''
    cursor.execute(delete_query, (id,))
    conn.commit()
    return 'OK', 200

@app.route('/update_product', methods=['POST'])
def update_product():
       data = request.get_json()
       id = data['id']
       name = data['name']
       description = data['description']
       price = data['price']
       conn = get_db()
       cursor = conn.cursor()
       cursor.execute('UPDATE products SET name = ?, description = ?, price = ? WHERE id = ?', (name, description, price, id))
       conn.commit()
       return 'Product updated', 200

@app.route('/delete_product', methods=['POST'])
def delete_product():
    data = request.get_json()
    id = data['id']
    conn = get_db()
    cursor = conn.cursor()
    cursor.execute('DELETE FROM products WHERE id = ?', (id,))
    conn.commit()
    return 'Product deleted', 200

@app.route('/list_products')
def list_products():
  conn = get_db()
  cursor = conn.cursor()
  cursor.execute('SELECT * FROM products')
  products = cursor.fetchall()
  return render_template('list_products.html', products=products)

@app.route('/add_product', methods=['POST'])
def add_product():
  data = request.get_json()
  name = data['name']
  description = data['description']
  price = data['price']
  conn = get_db()
  cursor = conn.cursor()
  cursor.execute('INSERT INTO products (name, description, price) VALUES (?, ?, ?)', (name, description, price))
  conn.commit()
  return 'Product added', 200



if __name__ == '__main__':
    app.run()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://3ae5-34-125-229-135.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [07/Jan/2024 08:09:37] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Jan/2024 08:09:38] "GET /static/js/scripts.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Jan/2024 08:09:39] "GET /static/images/favicon.ico HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Jan/2024 08:09:39] "GET /static/js/scripts.js HTTP/1.1" 200 -
